In [26]:
from __future__ import print_function
import argparse
import sys 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [27]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [28]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:
def mainAA():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    args = parser.parse_args()
    #use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_cuda = False 
    
    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)
    

    print("inside main function 3")
    
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)



In [30]:
print("hello world, work started ")
sys.argv = ['foo']
mainAA()

hello world, work started 
inside main function 2
inside main function 3
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332586
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.323807
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.262323
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.307054
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.248528
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.214086
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.219406
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.205292
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.088662
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.981406
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.929761
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.844703
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.841034
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.777270
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.767565
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.510256
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.443800
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.443561
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.388477
Train 

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.233563
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.491278
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.251761
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.271500
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.245959
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.182084
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.211336
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.292796
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.331125
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.315301
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.221098
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.246832
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.277857
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.418653
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.363318
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.398999
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.343617
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.406028
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.296691
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.261476


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.176459
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.242841
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.387468
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.147852
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.281705
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.152268
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.467391
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.275554
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.574346
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.328699
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.306291
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.280490
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.134561
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.372659
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.315350
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.381053
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.242055
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.347024
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.129867
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.318917


Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.217641
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.233918
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.216994
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.223732
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.228196
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.174462
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.156699
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.206589
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.189777
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.189027
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.187723
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.193439
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.368199
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.324917
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.236942
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.416159
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.163570
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.218667
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.300210
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.120047


Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.141060
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.167450
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.325548

Test set: Average loss: 0.0616, Accuracy: 9809/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.078968
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.107504
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.104977
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.067516
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.097333
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.232006
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.087875
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.149856
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.191705
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.293053
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.144606
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.083965
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.251827
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.157668
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.222149
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.234851
Train Epoch: 8 [10

Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.161394
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.069401
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.238885
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.161040
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.224818
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.126792
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.149366
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.168918
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.174046
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.182999
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.087668
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.125394
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.100501
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.226258
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.107994
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.196476
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.075918
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.145061
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.116701
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.179666
